In [31]:
#######################################################################
#
#
#  This program delens multiple images to the source plane using the file delens_data.dat
#  Suitable for doing delens for all the images in the cluster at one program run
#  This program only uses the column:
#     'source', 'objID', 'x_centroid', 'y_centroid', 'redshift', 'delens_start', 'delens_stop'
#  in the image_parameters file
#
#
#######################################################################


import pandas as pd
import pyfits
import numpy as np
import seaborn as sns
from scipy import interpolate
import math
from __future__ import division
import functools


from astropy.cosmology import FlatLambdaCDM
cosmo = FlatLambdaCDM(H0=70., Om0=0.3)
from astropy import constants as const


import matplotlib.pyplot as plt
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 12}
plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)

#%matplotlib inline


In [32]:
step = 0.05      # redshift resolution of the graph

Pix_ratio = 3173304.711     # in full resolution, how many pixels are there in 1 radian?
Npix = int(2048)     #field of view full resolution
Npix0 = int(512)     #input alpha file dimension
ver = 'ver22'
image_parameters_dir = '/Volumes/BRIAN/Research/CLASH/MACS0647/image_parameters.csv' 
#delens_data_dir = '/Volumes/BRIAN/Research/delens/MACS0647/ver22/delens_data_geometric_6.dat' 
#alphaX_512_rad_dir = '/Volumes/BRIAN/Research/WSLAP_results/Initial_error/%s/reconstruction/recomp_alpha_x_rad_ver30.fits' %ver
#alphaY_512_rad_dir = '/Volumes/BRIAN/Research/WSLAP_results/Initial_error/%s/reconstruction/recomp_alpha_y_rad_ver30.fits' %ver
alphaX_512_rad_dir = '/Volumes/BRIAN/Research/WSLAP_works/data/M0647/%s/reconstruction/recomp_alpha_x_rad_%s.fits' %(ver, ver)
alphaY_512_rad_dir = '/Volumes/BRIAN/Research/WSLAP_works/data/M0647/%s/reconstruction/recomp_alpha_y_rad_%s.fits' %(ver, ver)
#Img_dir = '/Volumes/BRIAN/Research/CLASH/MACS0647/cropped_images/macs0647_cropped_jpeg_r.fits'
#outfile_dir = '/Volumes/BRIAN/Research/delens/MACS0647/Photoz_error/' + ver +'/real_z/system6/'

#cosmic_weight parameters
z_fid = float(3.0)   #redshift of the fiducial field
z_lens = float(0.584)    #redshift of the lens

Omega = float(0.3)
Lambda = float(0.7)
Omega_k = float(1.0 - Omega - Lambda)
Dh = float(3.0e3)


In [33]:
###read the alphax file
hdulist = pyfits.open(alphaX_512_rad_dir)
alphaX_fid_512_rad = (hdulist[0].data)

###read the alphaY file
hdulist = pyfits.open(alphaY_512_rad_dir)
alphaY_fid_512_rad = (hdulist[0].data)


#================================================================== 
# = * = * = * = * = * = * = * = * = * = * = * = * = * = * = * 
# 
#   Read the image_parameters file
#   
# = * = * = * = * = * = * = * = * = * = * = * = * = * = * = * 
image_parameters = pd.read_csv(image_parameters_dir)
image_parameters['source'] = image_parameters['source'].astype(str)    # change the source ID to string format
image_parameters['objID'] = image_parameters['objID'].astype(str)    # change the object ID to string format
image_parameters = image_parameters.set_index(image_parameters['objID'])  # use object ID to be the row index
#print image_parameters


In [34]:
def decorator(d):
    "Make function d a decorator: d wraps a function fn."
    def _d(fn):
        return functools.update_wrapper(d(fn), fn)
    return _d
decorator = decorator(decorator)

@decorator
def memo(f):
    """Decorator that caches the return value for each call to f(args).
    Then when called again with same args, we can just look it up."""
    cache = {}
    def _f(*args):
        try:
            return cache[args]
        except KeyError:
            result = f(*args)
            cache[args] = result
            return result
        except TypeError:
            # some element of args can't be a dict key
            return f(*args)
    _f.cache = cache
    return _f

@memo
def cosmic_weight(z_src):
    if z_lens >= z_fid or z_lens >= z_src:
        print 'Error: z_lens must be smaller than z_fid or z_src.'
    Df = cosmo.angular_diameter_distance(z_fid)
    Dls = cosmo.angular_diameter_distance_z1z2(z_lens, z_src)
    Dlf = cosmo.angular_diameter_distance_z1z2(z_lens, z_fid)
    Ds = cosmo.angular_diameter_distance(z_src)
    weight = (Df * Dls)/(Dlf * Ds)
    #print'Redshift:   ' + str(z_src)
    #print'*** Cosmic Weight *** :   ' + str(weight)
    return weight

In [35]:
# = * = * = * = * = * = * = * = * = * = * = * = * = * = * = * 
# 
# Interpolate alpha
#   
# = * = * = * = * = * = * = * = * = * = * = * = * = * = * = * 
print(' Interpolating alpha ... ')
#alpha_X
x = np.arange(Npix0)
y = np.arange(Npix0)
fX = interpolate.interp2d(x, y, alphaX_fid_512_rad)

ratio = (Npix0)/Npix
xnew = np.arange(0,Npix0,ratio)
ynew = np.arange(0,Npix0,ratio)
alphaX_fid_ACS_rad = fX(xnew, ynew)

print('alphaX has dimension:' + str(alphaX_fid_ACS_rad.shape))

#alpha_Y
fY = interpolate.interp2d(x, y, alphaY_fid_512_rad)
alphaY_fid_ACS_rad = fY(xnew, ynew)

print('alphaY has dimension:' + str(alphaY_fid_ACS_rad.shape))

print(' Interpolation Done')

alphaX_fid_ACS = alphaX_fid_ACS_rad*Pix_ratio
alphaY_fid_ACS = alphaY_fid_ACS_rad*Pix_ratio

 Interpolating alpha ... 
alphaX has dimension:(2048, 2048)
alphaY has dimension:(2048, 2048)
 Interpolation Done


In [36]:
# = * = * = * = * = * = * = * = * = * = * = * = * = * = * = * 
# 
# Define a function that delens a point (unique fo this program)
#   
# = * = * = * = * = * = * = * = * = * = * = * = * = * = * = * 

def delens_pt(x, y, z):
    Ds_weight = cosmic_weight(z)
    alphaX_ACS = alphaX_fid_ACS * Ds_weight
    alphaY_ACS = alphaY_fid_ACS * Ds_weight
    i = x - int(round(alphaX_ACS[y,x]))
    j = y - int(round(alphaY_ACS[y,x]))
    return (i,j)

In [37]:
# = * = * = * = * = * = * = * = * = * = * = * = * = * = * = * 
# 
# Real work starts at here.
#   
# = * = * = * = * = * = * = * = * = * = * = * = * = * = * = * 

sources_list = map(str, list(set(image_parameters['source'].astype(int))))   # create a list of all sources
source_dispersion = []

for source in sources_list:
    single_source_df = image_parameters.groupby('source')['x_centroid', 'y_centroid', 'redshift', 'delens_start', 'delens_stop'].get_group(source)
    if single_source_df['delens_start'].std() != 0 or single_source_df['delens_stop'].std() != 0:
        print ' *** Inconsistent delens_start and delens_stop. *** '
        #quit()
    z_start = single_source_df.iloc[0]['delens_start']
    z_stop = single_source_df.iloc[0]['delens_stop']
    N_image = single_source_df.shape[0]
    z_bins = np.arange(z_start, z_stop + step, step)    # get an array of all delens redshifts
    #create a 2D array to store the delens redshifts and source dispersion
    #first column:    redshift
    #second column:   SD
    #third:           mean
    dispersion = np.zeros((z_bins.shape[0],3))
    for k in np.arange(len(z_bins)):
        # create a 2D array source_positions to store the source positions and dispersion
        #first column:    x-coordinate of delensed source
        #second column:   y-coordinate
        #third column:    distance to average position
        source_positions = np.zeros((N_image,3))  
        i = 0      #an index to iterate through source_positions
        #obtain the delensed source positions
        for image in single_source_df.index:
            x_centroid = single_source_df.ix[image]['x_centroid']
            y_centroid = single_source_df.ix[image]['y_centroid']
            (x_source, y_source) = delens_pt(x_centroid,y_centroid,z_bins[k])
            source_positions[i,0] = x_source
            source_positions[i,1] = y_source
            i += 1
        #find the average position of the delensed sources
        x_avg = np.mean(source_positions[:,0])
        y_avg = np.mean(source_positions[:,1])
        #calculate the dispersions of the delensed sources
        for j in range(N_image):
            x_source = source_positions[j,0]
            y_source = source_positions[j,1]
            D = math.sqrt((x_source-x_avg) ** 2 + (y_source-y_avg) ** 2)
            source_positions[j,2] = D
        dispersion[k,0] = z_bins[k]
        dispersion[k,1] = np.sum(source_positions[:,2])
        dispersion[k,2] = np.mean(source_positions[:,2])
    #find the dispersion at the input redshift
    input_redshift_index = int((single_source_df.iloc[0]['redshift'] - z_start) / step)
    D_at_input_redshift = dispersion[input_redshift_index, 1]
    source_dispersion.append(D_at_input_redshift)
    #######################################################
    minimum_D_position = dispersion[:,2].argmin()
    z_minimum_D = dispersion[minimum_D_position, 0]
    print ' *** Source %s ***' %source
    print 'Minimum dispersion at z = %s' %z_minimum_D
    print 'Dispersion at input z (%s) is: %s' %(single_source_df.iloc[0]['redshift'], D_at_input_redshift)
    print ''
    
    ####################################################
    #
    #   GRAPH PLOTTING
    #
    ####################################################
    
    z = dispersion[:,0]
    mean = dispersion[:,2] * 0.065   # change y-axis to arcseconds
    plt.rc('font', **font)
    plt.figure(figsize=(6,3.705),dpi=150)
    plt.plot(z, mean)
    fig = plt.gcf()
    plt.tight_layout(pad=2, w_pad=2, h_pad=2)  # a small space buffer at the margin
    #fig.set_size_inches(10,8, forward=True)
    #fig.set_dpi(150)
    ax = plt.gca()
    plt.axis([z_start, z_stop, 0, 1.5])
    # an extra feature to plot a transparent block on the graph
    ax.axvspan(10, 10.8, alpha=0.5, color='green')
    ##########
    plt.xlabel(r'Delens Redshifts ($z$)', fontsize = 12)
    plt.ylabel('Delens Source Dispersion (arcseconds)', fontsize = 12)

    #plt.show()

print 'The total dispersion of all sources at input redshift: %s' %sum(np.asarray(source_dispersion))
        

 *** Source 1 ***
Minimum dispersion at z = 2.15
Dispersion at input z (2.2) is: 7.08919998782

 *** Source 2 ***
Minimum dispersion at z = 4.55
Dispersion at input z (4.7) is: 6.0

 *** Source 3 ***
Minimum dispersion at z = 2.85
Dispersion at input z (3.1) is: 3.60555127546

 *** Source 4 ***
Minimum dispersion at z = 1.9
Dispersion at input z (1.9) is: 13.1379467878

 *** Source 5 ***
Minimum dispersion at z = 6.45
Dispersion at input z (6.5) is: 3.0

 *** Source 6 ***
Minimum dispersion at z = 9.8
Dispersion at input z (11.0) is: 10.3886807763

 *** Source 7 ***
Minimum dispersion at z = 2.2
Dispersion at input z (2.2) is: 5.71247021714

 *** Source 8 ***
Minimum dispersion at z = 2.1
Dispersion at input z (2.3) is: 2.2360679775

 *** Source 9 ***
Minimum dispersion at z = 6.1
Dispersion at input z (5.9) is: 9.01846171271

The total dispersion of all sources at input redshift: 60.1883787347
